In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from LineByLineNLPTextDataset import LineByLineNLPTextDataset
from nlp.builder import FORCE_REDOWNLOAD

from transformers import BertForMaskedLM, BertTokenizer

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
PyTorch version 1.6.0 available.
Apache Beam available.


In [2]:
OUTPUT_DIR = f"../../weights/es-lstm/"
TRAIN_PATH = f"../../data/minicorpus1/train_1.txt"
TEST_PATH = f"../../data/minicorpus1/test_1.txt"
VAL_PATH = f"../../data/minicorpus1/val_1.txt"

In [3]:
!ls ../../data/minicorpus1

test_1.txt  train_1.txt  val_1.txt


In [4]:
block_size = 64

In [5]:
tokenizer = BertTokenizer.from_pretrained("../../weights/beto/", do_lower_case=False)

In [6]:
dataset = LineByLineNLPTextDataset(
    tokenizer=tokenizer,
    file_path=TRAIN_PATH,
    block_size=block_size,
    overwrite_cache=False
)

https://s3.amazonaws.com/datasets.huggingface.co/nlp/datasets/text/text.py not found in cache or force_download set to True, downloading to /home/camilojd/.cache/huggingface/datasets/tmpibe9i_xw


storing https://s3.amazonaws.com/datasets.huggingface.co/nlp/datasets/text/text.py in cache at /home/camilojd/.cache/huggingface/datasets/3e34209a2741375a1db1ff03bf1abba1a9bd0e6016912d3ead0114b9d1ca2685.964a0948391d87479f79716750233e9a593a71bb70d468a93367cd00247e6451.py
creating metadata file for /home/camilojd/.cache/huggingface/datasets/3e34209a2741375a1db1ff03bf1abba1a9bd0e6016912d3ead0114b9d1ca2685.964a0948391d87479f79716750233e9a593a71bb70d468a93367cd00247e6451.py


Checking /home/camilojd/.cache/huggingface/datasets/3e34209a2741375a1db1ff03bf1abba1a9bd0e6016912d3ead0114b9d1ca2685.964a0948391d87479f79716750233e9a593a71bb70d468a93367cd00247e6451.py for additional imports.
Found main folder for dataset https://s3.amazonaws.com/datasets.huggingface.co/nlp/datasets/text/text.py at /home/camilojd/Universidad/ALS/nlp/src/nlp/datasets/text
Found specific version folder for dataset https://s3.amazonaws.com/datasets.huggingface.co/nlp/datasets/text/text.py at /home/camilojd/Universidad/ALS/nlp/src/nlp/datasets/text/c3b177069f0fad4da737a020bb39bbdb7aa16992e1f401e4347568618c906e28
Found script file from https://s3.amazonaws.com/datasets.huggingface.co/nlp/datasets/text/text.py to /home/camilojd/Universidad/ALS/nlp/src/nlp/datasets/text/c3b177069f0fad4da737a020bb39bbdb7aa16992e1f401e4347568618c906e28/text.py
Couldn't find dataset infos file at https://s3.amazonaws.com/datasets.huggingface.co/nlp/datasets/text/dataset_infos.json
Found metadata file for dataset

Dataset not on Hf google storage. Downloading and preparing it from source
Unable to verify checksums.
Generating split train


Done writing 2145301 examples in 479008370 bytes /home/camilojd/.cache/huggingface/datasets/text/dataset-d8cb74a71389f7f3/0.0.0/c3b177069f0fad4da737a020bb39bbdb7aa16992e1f401e4347568618c906e28.incomplete/text-train.arrow.
Unable to verify splits sizes.
Constructing Dataset for split train, from /home/camilojd/.cache/huggingface/datasets/text/dataset-d8cb74a71389f7f3/0.0.0/c3b177069f0fad4da737a020bb39bbdb7aa16992e1f401e4347568618c906e28
Unable to verify checksums.
Caching processed dataset at /home/camilojd/.cache/huggingface/datasets/text/dataset-d8cb74a71389f7f3/0.0.0/c3b177069f0fad4da737a020bb39bbdb7aa16992e1f401e4347568618c906e28/cache-ef661e41ba6d3417.arrow


Dataset text downloaded and prepared to /home/camilojd/.cache/huggingface/datasets/text/dataset-d8cb74a71389f7f3/0.0.0/c3b177069f0fad4da737a020bb39bbdb7aa16992e1f401e4347568618c906e28. Subsequent calls will reuse this data.


Done writing 2145301 examples in 2620006586 bytes /home/camilojd/.cache/huggingface/datasets/text/dataset-d8cb74a71389f7f3/0.0.0/c3b177069f0fad4da737a020bb39bbdb7aa16992e1f401e4347568618c906e28/tmp5wad97d6.
Set __getitem__(key) output type to torch for ['input_ids'] columns  (when key is int or slice) and don't output other (un-formatted) columns.


In [8]:
class GenerativeLSTM(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(GenerativeLSTM, self).__init__()
        # Params
        self.lstm_size = 128
        self.embedding_dim = 128
        self.hidden_size = hidden_size
        self.num_layers = 3 
        
        # Layers
        self.embedding = nn.Embedding(
            num_embeddings=output_size,
            embedding_dim=self.embedding_dim
        )
        
        self.lstm = nn.LSTM(
            input_size=self.lstm_size, 
            hidden_size=self.hidden_size, 
            num_layers=self.num_layers, 
            #bidirectional=True, 
            batch_first=True,
            dropout=0.2
        )
        
        self.fc = nn.Linear(self.hidden_size, output_size)
        
    def forward(self, input_ids, labels):
        embed = self.embedding(input_ids)
        lstm_output, lstm_state = self.lstm(embed)
        lm_logits = self.fc(lstm_output)
        
        outputs = (lm_logits,) + (lstm_output,)
        
        if labels is not None:
            # Shift so that tokens < n predict n
            shift_logits = lm_logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            # Flatten the tokens
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            outputs = (loss,) + outputs
        return outputs

In [9]:
hidden_size, output_size = 128 , len(tokenizer)
model = GenerativeLSTM(hidden_size, output_size)

In [10]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    per_device_train_batch_size=128,
    save_steps=10_000,
    save_total_limit=2,
    warmup_steps=10_000,
    max_steps=900_000,
    learning_rate=0.0001,
    #fp16=True
)

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False, mlm_probability=0.15
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

In [11]:
%%time
trainer.train()

/home/camilojd/Universidad/ALS/nlp/src/nlp/arrow_dataset.py:702: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)


ValueError: Trainer.model appears to not be a PreTrainedModel